In [74]:
import requests
import tensorflow as tf
import json
import base64
from pprint import PrettyPrinter

In [75]:
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/clickbait-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1672560275'}]}


In [76]:
# Menggunakan salah satu judul yang termasuk clickbait
test_clickbait = 'Should I Get Bings'

In [77]:
def prepare_json(text):
    feature_spec = {
        "headline": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")]))
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

In [78]:
def make_predictions(text):
    json_data = prepare_json(text)
    
    endpoint = "http://localhost:8080/v1/models/clickbait-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]
    return "Judul Clickbait" if prediction > 0.6 else "Judul Tidak Clickbait"

In [79]:
make_predictions(test_clickbait)

'Judul Clickbait'